In [1]:
#This code deals with finding the largest bicluster from 0-1 data
#backend used is remote
#NOTE : (with reference to the second half). The backend simulator (simulon_gaussian) on Xanadu-cloud couldn't compile the linear interferometor
# for the example question (a 30 x 30 adjacency matrix)
import numpy as np
import xcc
import strawberryfields as sf
from strawberryfields.apps import sample
from strawberryfields import decompositions
import networkx as nx

In [2]:
#setting xcc with api_key
xanadu_token = open("xanadu_token.txt", "r").read() #xanadu_token.txt does not really exist on github due to obvious reasons :) (you will have to get your own tokens by signing up to the Xanadu cloud)
settings = xcc.Settings(REFRESH_TOKEN=xanadu_token)
settings.save()

In [3]:
connection = xcc.Connection.load()
connection.ping().ok

True

# First, a demo code

In [40]:
# define the linear interferometer
#adapted code from: https://strawberryfields.ai/photonics/demos/run_gaussian_boson_sampling.html (original code uses local engine)
U = np.array([
 [ 0.219546940711-0.256534554457j, 0.611076853957+0.524178937791j,
    -0.102700187435+0.474478834685j,-0.027250232925+0.03729094623j],
 [ 0.451281863394+0.602582912475j, 0.456952590016+0.01230749109j,
    0.131625867435-0.450417744715j, 0.035283194078-0.053244267184j],
 [ 0.038710094355+0.492715562066j,-0.019212744068-0.321842852355j,
    -0.240776471286+0.524432833034j,-0.458388143039+0.329633367819j],
 [-0.156619083736+0.224568570065j, 0.109992223305-0.163750223027j,
    -0.421179844245+0.183644837982j, 0.818769184612+0.068015658737j]
])

# create the 4 mode Strawberry Fields program
gbs = sf.Program(4)

with gbs.context as q:
    # prepare the input squeezed states
    S = sf.ops.Sgate(1)
    S | q[0]
    S | q[1]
    S | q[2]
    S | q[3]

    # linear interferometer
    sf.ops.Interferometer(U) | q

In [41]:
eng = sf.Engine("gaussian")

In [42]:
shots = 10000
results = eng.run(gbs,shots=shots)

In [43]:
results

<Result: shots=0, num_modes=0, contains state=True>

In [44]:
# Fock states to measure at output
measure_states = [[0,0,0,0], [1,1,0,0], [0,1,0,1], [1,1,1,1], [2,0,0,0]]

# extract the probabilities of calculating several
# different Fock states at the output, and print them to the terminal
for i in measure_states:
    prob = results.state.fock_prob(i)
    print("|{}>: {}".format("".join(str(j) for j in i), prob))

|0000>: 0.17637844761413482
|1100>: 0.06855956371224504
|0101>: 0.0020560972589722904
|1111>: 0.00834294639988194
|2000>: 0.0103129452534403


# Now our code

In [34]:
eng = sf.RemoteEngine("simulon_gaussian",connection)

In [35]:
#Let us generate a problem, very simple one
#Total data size of 25 x 6
dataset = np.zeros((25,5))
#Embed a bicluster (contiguous)
dataset_ones = np.ones((5,5))
#Nest the bicluster in the data
dataset[5:10,:] = dataset_ones

In [36]:
#create an adjacency matrix out of this
#generate zeros
dataset_adj_matrix = np.zeros((dataset.shape[0]+dataset.shape[1],dataset.shape[0]+dataset.shape[1]))
#convert data into adjacency matrix
for i in range(dataset.shape[0]):
    for j in range(dataset.shape[1]):
        dataset_adj_matrix[i,j+dataset.shape[0]] = dataset[i,j]
#Create symmetry
dataset_adj_matrix = dataset_adj_matrix + dataset_adj_matrix.T

In [37]:
from strawberryfields.apps import sample
modes = dataset_adj_matrix.shape[0] #it is okay for the modes to be odd, but the submatrix would be even
n_mean = dataset_adj_matrix.shape[0]/2 #trick from clustering 
shots = 1000

In [38]:
mean_photon_per_mode = n_mean / float(modes)
p = sf.Program(modes)

In [39]:
#This is inspired by code on : https://strawberryfields.readthedocs.io/en/stable/_modules/strawberryfields/apps/sample.html
#Has been edited
with p.context as q:
        #Convert matrix A into gates
        sf.ops.GraphEmbed(dataset_adj_matrix, mean_photon_per_mode=mean_photon_per_mode) | q

        #Threshold based measurement
        sf.ops.MeasureThreshold() | q
        

In [40]:
s = eng.run(p, shots=shots)

2023-12-18 19:15:11,113 - INFO - Compiling program for device simulon_gaussian using compiler gaussian.
2023-12-18 19:16:18,960 - ERROR - The remote job a7920073-c707-492e-b22b-7c478d289c91 failed due to an internal server error: {'error-code': 'simulator-timeout', 'error-detail': 'The job did not complete in time. Please consider reducing the complexity of the job before trying again.'}. Please try again.


FailedJobError: The remote job a7920073-c707-492e-b22b-7c478d289c91 failed due to an internal server error: {'error-code': 'simulator-timeout', 'error-detail': 'The job did not complete in time. Please consider reducing the complexity of the job before trying again.'}. Please try again.

In [ ]:
#This is an issue, Thus, we used the sample method sf.apps on a compute server in our university